Blah blah calculating statistics good 
reference https://berthub.eu/articles/posts/a-science-experiment-part-2/ (also let him know I'm referencing this, that's just polite)
also reference https://en.wikipedia.org/wiki/GC_skew

In [ ]:
# For its use in colab notebook
!pip install jbrowse-jupyter

In [ ]:
#in Google Colab, it seems that we can't use Flask 2.1.3, so we'll force it to use 1.1.4
#!pip install Flask==1.1.4

In [ ]:
# For its use in colab notebook
!pip install pandas
import pandas as pd

In [ ]:
#refseq_name = 'NZ_CP007470.1' #it would be nice to extract this from fasta or gff (which is what we do below when reading the fasta file line by line)

All of the data we are using came originally from the NCBI link above. Things I did to make our lives easier:
* Created a bgzipped and faidx indexed (SAMtools) copy of the fasta sequence
* Created a bgzipped and tabix indexed copy of the GFF (which contains the gene annotations)
* Created an unzipped copy of the fasta sequence (to make it easier for us to read)
* Plopped all of this data in a AWS S3 bucket with CORS enabled for public access

Here we are creating an empty JBrowse configuration option, telling it that it will be a linear genome view (LGV) as opposed to a circular genome view (CGV) and setting the assembly, telling where to find the indexed fasta file.

In [ ]:
base_url = 'https://s3.amazonaws.com/wormbase-modencode/test/h_influenzae/h_influenzae_477'

In [ ]:
fasta_url = base_url + '.fa' #unzipped--this is what we are going to read in this script

This is the "magic" for reading the contents from a URL.

In [ ]:
import requests

r = requests.get(fasta_url, stream=True)

if r.encoding is None:
  r.encoding = 'utf-8'

refseq_name = ''
seq = ''
firstline = True
for line in r.iter_lines(decode_unicode=True):
  if line:
    if firstline:
      space = line.index(" ")
      refseq_name = line[1:space]
      print(refseq_name)
      firstline = False
      continue
    if line.find(">") > -1: #stop after reading the first sequence
      break
    seq = seq + line.rstrip("\n")

print(len(seq))
    
    

In [ ]:
# don't actally need create here--that's only good for "build in" geneomes hg19, hg38
from jbrowse_jupyter import create, launch

In [ ]:
import jbrowse_jupyter
config = jbrowse_jupyter.JBrowseConfig(view="LGV",)
config.set_assembly(base_url + '.fa.gz', # bgzipped and then samtools faidx
                    name=refseq_name)

Here we are creating our first JBrowse 2 track from the GFF3 file I mentioned above. It's pretty straight forward, just giving it a url for the data and a name that will be displayed in JBrowse 2 (the "name") and a track_id, which is what we use to refer to it programatically below.

In [ ]:
#add gene track
geneGFFUrl = base_url + '.gff.gz'
config.add_track(geneGFFUrl, name="Genes",track_id="Genes")

Now we start some computation. The window and step are for calculating sliding averages: the window is how many bases we'll use to calculate an average and the step is how many bases we'll move the "center" of the window with each time through the loop. The sample_frequency is for items that we are calculating for every base, but only saving some of the time to keep the data structure smaller.

In [ ]:
window = 40 #window has to be an even number
step   = 20
sample_freq = 100

Frist read the sequence data into a variable called `seq`

In [ ]:
#firstline = True
#linecount=0

#seq = '' #first try just putting all of the seq into a variable (rather than working on streaming sequence)

#tw = io.TextIOWrapper(r)
#while True:
#for line in io.TextIOWrapper(r):
#  rline = tw.readline()
#  if not rline:
#    break
#  if firstline:
#    firstline = False
#    continue # skip the header line
#  if rline.find(">") > -1: #assuming there are other sequences in the fasta, stop after the first one
                           # note that this is a bit of a hack here: the interaction between streaming the data from AWS and TextIOWrapper doesn't
                           # work great (that is, it closes before we read to the end), so I hacked the fasta file that we are reading to have two 
                           # copies of the bacterial genome it it
#    break
#  seq = seq + rline.rstrip("\n")

Now we are going to do the calculations that happen on every base (rather than sliding averages, that we'll do below). Two things are getting calculated here:
* The cummulative GC skew (basically, every time you see a G, add 1, every time you see a C, subtract 1)
* The total of each type of base
Notice that typically the number of As is typically roughly equal to the number of Ts and the same is true for Gs and Cs. I don't know if an accepted hypothosis for why that is has ever been presented.

A brief comment about the data structure we are building: it is essentially several "parallel" arrays (ie, the first element of each array constitutes one "point" and so on). Since we are working on one type of data for each data structure and on a single chromosome, the reference name and "data name" fields will be the same for each point, which feels a little silly, but gives us the flexibility to create data structures where that isn't the case.

In [ ]:
all_sum = {}
cummulative_skew = {}
sampled_skew = {}
cummulative_name = {}
start = {}
end   = {}
refName = {}
skew_name = {}
firstBase = True
i = 1 #start counting bases at 1 (yeah, I know)
for bp in seq: #loop over every base in the sequence
  if firstBase:
    if bp == "G":
      cummulative_skew[i] = 1
    elif bp == "C":
      cummulative_skew[i] = -1
    else:
      cummulative_skew[i] = 0
    firstBase = False
  # calculating cummulative skew at every base
  else:
    if bp == "G":
      cummulative_skew[i] = cummulative_skew[i-1] + 1
    elif bp == "C":
      cummulative_skew[i] = cummulative_skew[i-1] - 1
    else:
      cummulative_skew[i] = cummulative_skew[i-1]
  if i % sample_freq == 0:
    index = int(i/sample_freq)
    sampled_skew[index] = cummulative_skew[i]
    cummulative_name[index] = 'Cummulative GC skew'
    start[index] = i
    end[index] = i
    refName[index] = refseq_name
    skew_name[index] = 'Cummulative GC skew'
  #not really a frequency, really sum
  if bp in all_sum:
    all_sum[bp] += 1
  else:
    all_sum[bp] = 1
  i += 1
print(all_sum)

Create a pandas DataFrame for the skew data (this is the structure that JBrowse expects).

In [ ]:
skew_data = {'refName': refName,
                'start':start,
                'end':end,
                'name':skew_name,
                'score':sampled_skew}
skew_df = pd.DataFrame(skew_data)
print(skew_df)

Now to calculate the sliding average statistics, percent GC and local GC asymetry

In [ ]:
seqlen = len(seq)
GCpercent = {}
GCasym = {}
start = {}
end = {}
refName = {}
percent_name={}
asym_name={}
rowcount=0
for bp in range(int(window/2), int(seqlen-window/2), step): #note the use of range to create a number sequence to skip through the sequence string
  seqfrag = seq[bp-int(window/2):bp+int(window/2)]
  Acount = seqfrag.count('A')
  Ccount = seqfrag.count('C')
  Gcount = seqfrag.count('G')
  Tcount = seqfrag.count('T')
  refName[rowcount] = refseq_name
  start[rowcount] = bp
  end[rowcount] = bp
  percent_name[rowcount] = 'GC percent'
  asym_name[rowcount] = 'GC asymetry'
#casting fractions into int percents; score shouldn't have to be an int!
  GCpercent[rowcount] = int((100*Gcount+Ccount)/window)
  GCasym[rowcount] = 0
  if Gcount+Ccount > 0:
    GCasym[rowcount]  = int(100*(Gcount-Ccount)/(Gcount+Ccount))
  rowcount = rowcount+1
    


  

And create pandas DataFrames for percent GC and GC asymetry data

In [ ]:
percent_data = {'refName': refName,
                'start':start,
                'end':end,
                'name':percent_name,
                'score':GCpercent}
percent_df = pd.DataFrame(percent_data)
print(percent_df)

In [ ]:
asym_data = {'refName': refName,
                'start':start,
                'end':end,
                'name':asym_name,
                'score':GCasym}
asym_df = pd.DataFrame(asym_data)
print(asym_df)

I'm printing out the config just to get a look at it to make sure it looks ok before I start adding data (which will make the config really big)

In [ ]:
print(config.get_config())

In [ ]:
config.add_df_track(percent_df,'percent GC',track_id='df_percent_gc',overwrite=True)
config.add_df_track(asym_df,'GC asymetry',track_id='df_gc_asym',overwrite=True)
config.add_df_track(skew_df,'Cummulative GC skew',track_id='df_gc_skew',overwrite=True)

Finally, we set a few things about how we want the initial view of JBrowse to look (location, tracks that are turned on) and then launch the genome browser. Note that as we zoom, it might get a little "jerky," as the config file is fairly large, since all of the data we created above is in the config. If we wanted to make this into something we showed other people, we'd want to create data files for each of these tracks.

In [ ]:
location = refseq_name + ":1000..3500"
print(location)


In [ ]:
config.set_location(location)
#config.set_default_session(['Genes','df_gc_skew','df_percent_gc','df_gc_asym'], False)
config.set_default_session(['Genes','df_gc_skew'], False)
full_conf = config.get_config()
launch(full_conf, port=3003, height=600) #height sets the height of the iframe containing JBrowse, but 
                                         # JBrowse sets the height of itself. To use all of the height allocated,
                                         # you have to manually drag the bottom edge of jbrowse down.